In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb
import os
import matplotlib.font_manager as fm
import yfinance as yf
import datetime
from datetime import timedelta, datetime
import openpyxl
import setuptools
import pandas_datareader as pdr

In [32]:
df_GARE13 = pd.read_excel('GARE13_22-08.xlsx')
GARE11_base = pd.read_excel('GARE11_18-08.xlsx')
df_GARE12 = pd.read_excel('GARE12_08.08.xlsx')

In [10]:
GARE11_base

,DOCUMENTO,NOME,PF_PJ,TOTAL_NOVO,FINANCEIRO_NOVO,CATEGORIAS ATUAIS
0,6556,GLEDSON ANDRADE SANTOS,PF,6200,56358.00,50k-100k
1,80527,HUGO LEONARDO LEMOS MATOS,PF,24,218.16,0-1k
2,114790,HANS HERBERT LAUBMEYER FILHO,PF,360,3272.40,1k-5k
3,200018,VERIDIANE CRISTINA ZACARIA,PF,2,18.18,0-1k
4,549258,NICOLE PEREIRA DA COSTA,PF,5,45.45,0-1k
...,...,...,...,...,...,...
394618,99346770678,CLAUDIANO TABORDA COELHO,PF,10,90.90,0-1k
394619,99357720120,SERGIO LUCENA RESENDE,PF,1,9.09,0-1k
394620,99452391200,THIAGO ROCHA DE MENEZES,PF,1,9.09,0-1k
394621,99830507068,RODRIGO PIRES ACOSTA,PF,109,990.81,0-1k


In [33]:
df_analise = df_GARE13
df_analise['COTISTA_GARE11'] = df_analise['DOCUMENTO'].apply(lambda doc: 'GARE11' if doc in GARE11_base['DOCUMENTO'].values else 'AUSENTE')
df_analise['COTISTA_GARE12'] = df_analise['DOCUMENTO'].apply(lambda doc: 'GARE12' if doc in df_GARE12['DOCUMENTO'].values else 'AUSENTE')
df_analise_merge = pd.merge(df_GARE13, GARE11_base[['DOCUMENTO', 'CATEGORIAS ATUAIS']],
                            on = ['DOCUMENTO'],
                            how = 'left',
                            suffixes=('', '_GARE11'))
df_analise_merge['CATEGORIAS_ATUAIS'] = df_analise_merge['CATEGORIAS ATUAIS'].fillna(0)

In [30]:
gare13_cat = df_analise_merge.groupby(['CATEGORIAS_ATUAIS'])['TOTAL'].sum()
gare13_cat_sorted = gare13_cat.sort_values(ascending=False)
gare13_dataframe = pd.DataFrame(gare13_cat_sorted)
gare13_dataframe = gare13_dataframe.reset_index()
gare13_dataframe

,CATEGORIAS_ATUAIS,TOTAL
0,0,393335
1,>15M,353715
2,100k-500k,258041
3,20k-50k,186125
4,5k-20k,170531
5,50k-100k,159206
6,1k-5k,129805
7,0-1k,63825
8,500k-1M,32336


In [31]:
gare13_cat = df_analise_merge.groupby(['CATEGORIAS_ATUAIS'])['TOTAL'].count()
gare13_cat_sorted = gare13_cat.sort_values(ascending=False)
gare13_dataframe = pd.DataFrame(gare13_cat_sorted)
gare13_dataframe = gare13_dataframe.reset_index()
gare13_dataframe

,CATEGORIAS_ATUAIS,TOTAL
0,0-1k,5353
1,1k-5k,2067
2,5k-20k,730
3,0,339
4,20k-50k,268
5,50k-100k,84
6,100k-500k,53
7,500k-1M,4
8,>15M,1


In [34]:
df_ausente = df_analise_merge.query('COTISTA_GARE11 == "AUSENTE"')
df_ausente

,CHAVE,DOCUMENTO,NOME,PF_PJ,TOTAL,COTISTA_GARE11,COTISTA_GARE12,CATEGORIAS ATUAIS,CATEGORIAS_ATUAIS
2,00008799750ANDRE BAPTISTA BARCAUI,8799750,ANDRE BAPTISTA BARCAUI,PF,1,AUSENTE,AUSENTE,NaN,0
30,00126679193STEPHANIE MIRANDA IMBROISI CAP,126679193,STEPHANIE MIRANDA IMBROISI CAPONE,PF,6000,AUSENTE,GARE12,NaN,0
53,00227230647GLAUCIANA FERNANDES PEIXOTO DE,227230647,GLAUCIANA FERNANDES PEIXOTO DE SIQUEIRA,PF,1688,AUSENTE,AUSENTE,NaN,0
60,00255340680CARLA VILAS BOAS,255340680,CARLA VILAS BOAS,PF,5,AUSENTE,GARE12,NaN,0
104,00364109777ELIZABETH COHEN,364109777,ELIZABETH COHEN,PF,4354,AUSENTE,GARE12,NaN,0
...,...,...,...,...,...,...,...,...,...
8848,98104560778AUGUSTO CESAR TEIXEIRA LEITE,98104560778,AUGUSTO CESAR TEIXEIRA LEITE,PF,10,AUSENTE,GARE12,NaN,0
8861,98618652768CARLOS ALBERTO BOURGUIGNON,98618652768,CARLOS ALBERTO BOURGUIGNON,PF,1006,AUSENTE,AUSENTE,NaN,0
8862,98680196053ROBSON DIAS FLORES,98680196053,ROBSON DIAS FLORES,PF,50,AUSENTE,GARE12,NaN,0
8870,98888242953GILSON SAKUMA DE OLIVEIRA,98888242953,GILSON SAKUMA DE OLIVEIRA,PF,10,AUSENTE,AUSENTE,NaN,0


In [38]:
df_analise_merge.groupby(['COTISTA_GARE12'])['TOTAL'].count()

COTISTA_GARE12
AUSENTE     863
GARE12     8036
Name: TOTAL, dtype: int64